                                            Inicializador do Navegador

In [12]:
 # Criar um navegador 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

navegador = webdriver.Chrome()


                                                    Tabela Produtos

In [13]:
# Importar ee Visualizar a base de dados
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,1000,3500
1,rtx 3060,zota galax,1000,4500


                                                   PESQUISA BUSCAPÈ

In [14]:

import time

def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            termos_banidos = True
    return tem_termos_banidos
            
          
def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False 
    return tem_todos_termos_produtos

# busca pelo produto

produto = "iphone 12 64gb"
termos_banidos = "mini watch"
lista_termos_banidos = termos_banidos.split(" ")
lista_termos_nome_produto = produto.split(" ")
preco_minimo = 1000
preco_maximo = 5000
lista_resultados = []
lista_ofertas = []

# Função Buscapé
def busca_busca_pe(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    # tratar valores
  produto = produto.lower()
  termos_banidos = termos_banidos.lower()
  lista_termos_banidos = termos_banidos.split(" ")
  lista_termos_nome_produto = produto.split(" ")
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)
   
    # buscar no buscapé
  navegador.get("https://www.buscape.com.br/?og=19220&og=19220&gad_source=1&gclid=Cj0KCQjwy46_BhDOARIsAIvmcwOnHurHVyPH_3J8ZqatqvlRY-JPc2Wi8fJYVz5jvwcRQUVJSo6dMYwaAhFtEALw_wcB")
  time.sleep(2)
  navegador.find_element('xpath', '/html/body/div[1]/main/header/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

# pegar os resultados
  lista_resultados = WebDriverWait(navegador, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')))
  for resultado in lista_resultados:
      preco = WebDriverWait(resultado, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'Text_Text__ARJdp.Text_MobileHeadingS__HEz7L'))
              ).text
      nome = WebDriverWait(resultado, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'Text_Text__ARJdp.Text_MobileLabelXs__dHwGG.Text_DesktopLabelSAtLarge__wWsED.ProductCard_ProductCard_Name__U_mUQ'))
              ).text
      link = resultado.get_attribute('href') 
    #print(nome, preco, link) # pegar o link
    # analisar se ele não tem nenhum termo banido
      tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos,nome)

    #analisar se ele tem TODOS os termos do produto
      tem_todos_termos_produto = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)
# selecionar só os elementos que tem_temos_banidos = False e ao mesmo tempo tem_todos_temos_produto
      if not tem_termos_banidos and tem_todos_termos_produto:
          preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
          preco = float(preco)
        # analisar se o preço esta entre o preço minimo e preço maximo
          if not tem_termos_banidos and tem_todos_termos_produto:
              if preco_minimo <= preco <= preco_maximo:
                  lista_ofertas.append((nome, preco, link))
  return lista_ofertas


                                                Tabela com Produtos

In [15]:
tabela_ofertas = pd.DataFrame() # tabela que junta todos resultados

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_buscape = busca_busca_pe(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:   # verifica se tem alguma coisa na lista
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=["produto", "preco", "link"])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
else:
        tabela_buscape = None
        



                                                 Esportando para o Excel

In [5]:
tabela_ofertas.to_excel("Ofertas.xlsx", index=False) # index vai eliminar a coluna indice 

                                                   Enviando por E-mail

In [18]:
import email.message
import smtplib

if len(tabela_ofertas) > 0:
    # emails = pd.read_excel(r'tabela_ofertas.xlsx')  # Removed as tabela_ofertas is already available
    senha = 'trar omgq rpld ryja '
    msg = email.message.EmailMessage()
    msg['From'] = "ricardogropofx@gmail.com"
    msg['To'] = "recipient@example.com"  # Replace with the recipient's email address
    msg["Subject"] = "Tabela com Produtos Encontrados"
    msg.set_content(f"""
    Prezados Gerentes,

    Segue tabela com produtos encontrados.

    Att. Ricardo
    """)
    msg.add_alternative(f"""
    <html>
        <body>
            <p>Prezados Gerentes,</p>
            <p>Segue tabela com produtos encontrados:</p>
            {tabela_ofertas.to_html(index=False)}
            <p>Att. Ricardo</p>
        </body>
    </html>
    """, subtype='html')
    servidor = smtplib.SMTP('smtp.gmail.com', 587)
    servidor.starttls()
    servidor.login(msg["From"], senha)
    servidor.send_message(msg)
    servidor.quit()
    print('E-mail enviado com sucesso!')


E-mail enviado com sucesso!
